# A brief introduction to Python for Hydrological Analyses
<br>
<img style="float: left; padding-right: 15px; padding-left: 0px;" src="source/logo_flood_proofs.png" width="260px" align=”left” >

<div style="text-align: justify">This is a Jupyter Notebook, a web-based interactive development environment that allows to create and share python codes.
First things first, what is **Python**? Python is an high-level and general-purpose programming language. It can be used to write software in a wide variety of application domains, including hydrology. Python can be used to perform numerical calculations, statistical analyses or to access and plot data (even large datasets). <br>
In Jupyter Notebook the *Python shell* is embedded. The shell is where you can write and execute a line (or multiple lines) of code.
Python is open-source, and several packages are available covering many scientific and technological fields.</div>

Let's start using Python as a **calculator**:

In [2]:
190/3


63.333333333333336

If needed we can assign this result to a variable, and use the variable for further math or for other operations (such as converting to integer and priting it).

In [ ]:
result = 190/3
new_result = result - 14
int_result = int(new_result)
print(int(int_result))


What if we want to perform some more complex calculus? We can import the **math** package, loading several mathematical functions (such as the square root)

In [ ]:
import math
sqrt_result = math.sqrt(int_result)
print(sqrt_result)


What if we want to work not with a single value, but with a **vector** composed of multiple values?

In [ ]:
import numpy as np
array = [1,4,100,3,-2]
print(array)
print('------> complete array maximum: ' + str(np.max(array)))
array_nan = array
array_nan[2] = np.nan
print(array_nan)
print('------> incomplete array maximum: ' + str(np.max(array)))


Is the last result correct? Shouldn't be 4 the new maximum value? We can use a specific function for accounting for "Nan" or missing values: *np.nanmax* (part of numpy package)

In [ ]:
print('------> incomplete array maximum: ' + str(np.nanmax(array)))


### Dealing with Timeseries
Can we generate **timeseries** (multiple values with associated date) and plot it? Of course!
Let's start by importing some useful libraries!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import wget
import os

# We can also define personal function, this one for example allow to visualize the output of dropdowns menus
def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            print("Selected value: " + change['new'])


We can now proceed with a random timeseries!

In [ ]:
ts = pd.Series(np.random.randn(365), index=pd.date_range('1/1/2010', periods=365))
plt.figure()
ts.plot(style='b-', label='Random timeseries')
plt.legend()


Most interesting is to work with existing timeseries. The NOAA provides a global dataset of daily rainfall and temperature values, the **Global Historical Climate Network Daily** (https://www.ncdc.noaa.gov/ghcn-daily-description).
The dataset can be queried by country by selecting it from the following dropdown menu.

In [ ]:
# Load list of available countries and generate dropdown selector
with open('source/ghcnd-countries.txt', 'r') as file:
    list_country = [line for line in file]
country_chooser = widgets.Dropdown(
    options=['Choose a country'] + list_country,
    value='Choose a country',
    description='Country:',
    disabled=False,
)
country_chooser.observe(on_change)
display(country_chooser)


By executing the next piece of code a dropdown menu of the available station for the selected country will be show. Please, select the station of interest.
**NB!** Some countries (Brazil, Australia and US) have too many station and can break the system, for those countries manual selection of the station is feasible!

In [ ]:
# Generate the list of the available stations
country_code = country_chooser.value[0:2]
list_stations = pd.read_fwf('source/ghcnd-stations.txt',
                            widths=[2,9,9,10,7,4,31,3,10],
                            header=None, usecols=[0,1,2,3,4,6], 
                            names=['COUNTRY','CODE','LAT','LON','ELEV','NAME'])
list_stations_in = list_stations.loc[list_stations['COUNTRY']==country_code].sort_values('NAME', ascending=True)

if len(list_stations_in)<4000:
    station_chooser = widgets.Dropdown(
        options=['Choose a station'] + list(list_stations_in['COUNTRY'] + list_stations_in['CODE'] + ' ' + list_stations_in['NAME']),
        value='Choose a station',
        description='Station:',
        disabled=False,
    )
    station_chooser.observe(on_change)
    display(station_chooser)
else:
    print('Station list is too long! Please, manually choose the station code from the available list at the web address https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt among the ones with first column starting with ' + country_code)


The next piece of code will download the series from the noaa series and analyse the available variables. 
Choose the variable for the analysis from the dropdown menu:

In [ ]:
# Insert information only for manually selecting the station code (for Brazil, Austalia and US)
section_code =  None     # es: 'BR00B7-0400'
section_name = None      # es: 'SAO JOAO DE IRACEMA'
###############################################################################################

if section_code is None:
    section_code = station_chooser.value.split(' ', 1)[0]
    section_name = station_chooser.value.split(' ', 1)[1]
file_name = section_code +'.csv'
out_path = 'meteo/' + file_name

# Check if the file has been already downloaded
if os.path.isfile(out_path):
    print('Section ' + section_code + ' ' + section_name + ' already downloaded!')
    print('DONE!')
else:
    print('Dowloading section ' + section_code + ' ' + section_name + '... It can take some times!')
    https_address = 'https://www.ncei.noaa.gov/data/global-historical-climatology-network-daily/access/'
    wget.download(https_address + file_name, out = out_path)
    print('DONE!')
    
# Open the file and analyse the available variables
info_station = pd.read_csv(out_path, header=0, usecols=['STATION','NAME','LATITUDE','LONGITUDE','ELEVATION'], nrows=1)
full_series = pd.read_csv(out_path, header=0, index_col='DATE', parse_dates=True, usecols=lambda c: c in {'DATE','PRCP','SNWD','TMAX','TMIN','TAVG'}) #, usecols=[1,2,3], names=['date','type','val'])
dic_vars={'precipitation':['PRCP', 'Rainfall(mm)'], 'temperature mean':['TAVG','Temperature(°C)'], 'temperature max':['TMAX','Temperature(°C)'], 'temperature min':['TMIN','Temperature(°C)'], 'snow depth':['SNWD', 'Snow depth(cm)']}

available_vars = [i for i in dic_vars if dic_vars[i][0] in full_series.columns]

var_chooser = widgets.Dropdown(
    options=['Choose a variable'] + available_vars,
    value='Choose a variable',
    description='Vars available:',
    disabled=False,
)

var_chooser.observe(on_change)
display(info_station.style.hide_index())
display(var_chooser)


Now we can interactively plot one of the available timeseries by choosing the station, the variable and also the time limits: 

In [ ]:
# Insert information only for choosing a sub-period of the whole series
time_start = None    # Set a date in the format 'YYYY-MM-DD' or None for plot the series from the beginning
time_end = None      # Set a date in the format 'YYYY-MM-DD' or None for plot the series up to the end
####################################################################

# Read data series
variable = var_chooser.value
temp_series = full_series[[dic_vars[variable][0]]]/10

# Set time range
time_start = temp_series.first_valid_index() if time_start is None else pd.to_datetime(time_start,format='%Y-%m-%d')
time_end = temp_series.last_valid_index() if time_end is None else pd.to_datetime(time_end,format='%Y-%m-%d')
if time_start > time_end:
    raise ValueError("time_start is larger than time_end, verify your data!")
time_range = pd.date_range(time_start,time_end,freq='1D')
temp_series = temp_series.reindex(time_range)

display(temp_series)

# Manage plot
ax = temp_series.plot(style='b', title=variable + ' at ' + section_name, figsize=(15,5))
ax.set_xlabel("")
ax.set_ylabel(dic_vars[variable][1])
ax.get_legend().remove()
plt.show()


Time series can be asily managed with python for resampling and statistical operations:

In [ ]:
# The resample frequency can be set, e.g., to annual 'Y' or monthly 'M'
temp_resampled_max = temp_series.reindex(time_range).resample('M').max()
temp_resampled_min = temp_series.reindex(time_range).resample('M').min()
temp_resampled_avg = temp_series.reindex(time_range).resample('M').mean()

# Manage plot
ax = temp_resampled_max.plot(style='r', title=variable + ' at ' + section_name, figsize=(15,5))
temp_resampled_min.plot(style='b',ax=ax)
temp_resampled_avg.plot(style='g',ax=ax)
ax.set_xlabel("")
ax.set_ylabel(dic_vars[variable][1])
plt.legend(['max','min','avg'])
plt.show()


## Flood frequency analysis using Python
We can use python to compute flood statistics on a discharge timeseries. Reference to *hydro-informatics.github.io* <br>

Occurence of relevant (extreme) flood events can be expressed as **return period**, expressing the average recurrence interval of an event of a certain magnitude in units of time. It is the inverse of the **exceedance probability** (the likelihood of an event of a certain magnitude or higher).<br>
A significant assumption in calculating the return period is that individual events are assumed indipendent. This means that, for any given year, the probability of a 100-year flood occurring is 1/100.
Here below a table showing the recurrence intervals and related probabilities of occurrences.

| Return Period (years) | Annual exceeding probability (%) |
| --- | --- |
| 2 | 50 |
| 5 | 10 |
| 10 | 10 |
| 50 | 2 |
| 100 | 1 |
| 500 | 0.2 |

At first we should import some useful libraries:

In [ ]:
import numpy as np
import pandas as pd
import glob
import wget
import os
import zipfile


Then, we should import the **discharge data**. 
Let's see what "txt" series files can be find inside the "discharge" folder.
Further data can be downloaded from the GRDC data portal by performing a custom request (https://portal.grdc.bafg.de/applications/public.html?publicuser=PublicUser#dataDownload/Stations). The request will be evaluated by the data provider and in not much time an email with a download url will be provided. 
The url can be inserted and diectly analysed with this tool:

In [ ]:
## Modify this section by inserting the download address provided by the GRDC website for download more data
# es: download_link = 'https://portal.grdc.bafg.de/grdcdownload/external/53c13313-e359-4f61-bb9d-d803b2ab74e1/2021-07-01_16-52.zip'
download_link = None
############################################################################################################

if not download_link is None:
    os.makedirs('temp', exist_ok=True)
    wget.download(download_link, out= 'temp/')
    with zipfile.ZipFile('temp/' + os.path.basename(download_link), 'r') as zip_ref:
        zip_ref.extractall('discharge/')
    
files = glob.glob("discharge/*.txt")
print(files)


The list don't tell us much abouyt the file content, we can open one of them to understand the content of each file 

**NOTE! Python numbering starts from 0!!**:

In [ ]:
# Read a preview of the file
number_of_lines = 40

with open(files[0],'rb') as file:
    for i in np.arange(0,number_of_lines,1): 
        line = file.readline().decode('ISO-8859-1')
        print(str(i) + ' ' + line)


The lines between 8 and 18 of each file contains all the information about the station, we can use python capability of manage different file type to summarize those information in a table:

In [ ]:
# Read the 11 lines after line 8 (Python numbering starts from 0!)
for ind, file in enumerate(files,0):
    data = pd.read_csv(file, skiprows=8, nrows=11, sep=":", encoding='ISO-8859-1', header=None, names=['cod','val'])    
    if ind == 0:
        list_vars = [i.replace('# ','') for i in data['cod']]
        df_stations = pd.DataFrame(index=np.arange(0,len(files),1),columns=list_vars)
    data['cod'] = list_vars
    data = data.set_index(['cod'])
    for var in list_vars:
        df_stations.loc[ind][var] = data.loc[var].values[0].strip()

df_stations = df_stations.set_index(["GRDC-No."])
display(df_stations)


We can now choose which station to analyse by providing its code to identify the related file, let's start, for example, with the **AWASH WENZ at MELKA KUNTIRE** (COD: 1577100):

In [ ]:
# Please, specify an available station code
station_code = '1577100'
##############################################################################################

# Read data from line 37
df = pd.read_csv("discharge/" + station_code + "_Q_Day.Cmd.txt",
                 header=None,
                 sep=";",
                 skiprows=37,
                 names=["Date", "Time","Q"],
                 parse_dates=[0],
                 index_col=["Date"])
df['Q']=df['Q'].astype(float)
ax = df.plot(title=df_stations.loc[station_code]["River"] + " at " + df_stations.loc[station_code]["Station"], figsize=(15,5))
ax.set_ylabel('Q (m3/s)')


There are null values in the series, that corresponds to null values, we can mange them by repalacing with "np.nan" that is the standard numpy null value:

In [ ]:
# Replace negative values with "null"
df.loc[df['Q']<0,'Q']=np.nan
ax = df.plot(title=df_stations.loc[station_code]["River"] + " at " + df_stations.loc[station_code]["Station"], figsize=(15,5))
ax.set_ylabel('Q (m3/s)')


This is the complete timeseries: we have to select only the **yearly maxima**. It is quite straightforward with *pandas dataframe*, we can resample our dataset (which has been indexed with dates).


In [ ]:
# Resample using the annual maximum value
df_ymax = df.resample("Y").max()
df_ymax["year"] = df_ymax.index.year
df_ymax.reset_index(inplace=True, drop=True)
df_ymax = df_ymax.dropna()
print(df_ymax)

# Manage plot
df_ymax.plot(kind='scatter',x='year',y='Q',color='red', figsize=(15,5))
plt.show()


### Return period analysis
We should compute the exceedence probability *Pr*, and the resulting recurrence interval.
Pr is defined as: $Pr_{i} = \frac{(n-i+1)}{n+1}$\
Where *n* is the total number of observation years and *i* is the rank of the event.

In [ ]:
# Sort in increasing order
df_ymax_sorted = df_ymax.sort_values(by="Q")
n = df_ymax_sorted.shape[0]
df_ymax_sorted.insert(0, "rank", range(1, 1 + n))
print(df_ymax_sorted)


The **exceedence probability** ( *pr* ) can be calculated applying the formula:

In [ ]:
df_ymax_sorted["pr"] = (n - df_ymax_sorted["rank"] + 1) / (n + 1)
print(df_ymax_sorted.tail())


The **recurrence interval**( *return-period* ) is the inverse of the probability, thus:

In [ ]:
df_ymax_sorted["return-period"] = 1 / df_ymax_sorted["pr"]
print(df_ymax_sorted.tail())


Once create the table (*dataframe*) with all required information (**Probability** and **Return-Period**) we might plot it to visualise the recurrence interval of each observed discharge. It is worth mentioning that this analysis and the resulting plot refer only to observed values.<br>
To extrapolate recurrence interval beyond the observation period (the 1-in-100 years flood values, for instance) a prediction model is needed (Gumbel, GEV, etc..).

In [ ]:
df_ymax_sorted.plot.scatter(y="Q",
                         x="return-period",
                         title="Return period [years] ",
                         color='blue',
                         grid=True,
                         fontsize=14,
                         logy=False,
                         label="Sorted values",
                         figsize=(15,10))


## That is for this brief practical introduction to Python! 
### You might exercise a bit:

* in the virtual environment you should find other discharge .txt files. Ask python to read and analyse your selected input file!
* which is the discharge value for Return Period = 5?
* which is the discharge value with a Probability of exceedance = 0.5?
* can you plot the return-periods in a logaritmic scale (for y)?